In [1]:
import EncoderFactory
from DatasetManager import DatasetManager
import BucketFactory

import pandas as pd
import numpy as np

from sklearn.metrics import roc_auc_score
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import StandardScaler


import time
import os
import sys
from sys import argv
import pickle
from collections import defaultdict

from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import catboost

from hyperopt import Trials, STATUS_OK, tpe, fmin, hp
import hyperopt
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample


def create_and_evaluate_model(args):
    global trial_nr
    trial_nr += 1
    
    start = time.time()
    score = 0
    for cv_iter in range(n_splits):
        
        dt_test_prefixes = dt_prefixes[cv_iter]
        dt_train_prefixes = pd.DataFrame()
        for cv_train_iter in range(n_splits): 
            if cv_train_iter != cv_iter:
                dt_train_prefixes = pd.concat([dt_train_prefixes, dt_prefixes[cv_train_iter]], axis=0, sort=False)
        
        # Bucketing prefixes based on control flow
        bucketer_args = {'encoding_method':bucket_encoding, 
                         'case_id_col':dataset_manager.case_id_col, 
                         'cat_cols':[dataset_manager.activity_col], 
                         'num_cols':[], 
                         'random_state':random_state}
        if bucket_method == "cluster":
            bucketer_args["n_clusters"] = args["n_clusters"]
        bucketer = BucketFactory.get_bucketer(bucket_method, **bucketer_args)
        bucket_assignments_train = bucketer.fit_predict(dt_train_prefixes)
        bucket_assignments_test = bucketer.predict(dt_test_prefixes)
        
        preds_all = []
        test_y_all = []
        if "prefix" in method_name:
            scores = defaultdict(int)
        for bucket in set(bucket_assignments_test):
            relevant_train_cases_bucket = dataset_manager.get_indexes(dt_train_prefixes)[bucket_assignments_train == bucket]
            relevant_test_cases_bucket = dataset_manager.get_indexes(dt_test_prefixes)[bucket_assignments_test == bucket]
            dt_test_bucket = dataset_manager.get_relevant_data_by_indexes(dt_test_prefixes, relevant_test_cases_bucket)
            test_y = dataset_manager.get_label_numeric(dt_test_bucket)
            if len(relevant_train_cases_bucket) == 0:
                preds = [class_ratios[cv_iter]] * len(relevant_test_cases_bucket)
            else:
                dt_train_bucket = dataset_manager.get_relevant_data_by_indexes(dt_train_prefixes, relevant_train_cases_bucket) # one row per event
                train_y = dataset_manager.get_label_numeric(dt_train_bucket)
                
                if len(set(train_y)) < 2:
                    preds = [train_y[0]] * len(relevant_test_cases_bucket)
                else:
                    feature_combiner = FeatureUnion([(method, EncoderFactory.get_encoder(method, **cls_encoder_args)) for method in methods])

                    if cls_method == "rf":
                        cls = RandomForestClassifier(n_estimators=500,
                                                     max_features=args['max_features'],
                                                     random_state=random_state)

                    elif cls_method == "xgboost":
                        cls = xgb.XGBClassifier(objective='binary:logistic',
                                                n_estimators=500,
                                                learning_rate= args['learning_rate'],
                                                subsample=args['subsample'],
                                                max_depth=int(args['max_depth']),
                                                colsample_bytree=args['colsample_bytree'],
                                                min_child_weight=int(args['min_child_weight']),
                                                seed=random_state)
                    elif cls_method == "cb":
                        cls =  catboost.CatBoostClassifier(learning_rate= args['learning_rate'],
                                                           subsample=args['subsample'], depth=int(args['depth']))

                    elif cls_method == "logit":
                        cls = LogisticRegression(C=2**args['C'],
                                                 random_state=random_state)

                    elif cls_method == "svm":
                        cls = SVC(C=2**args['C'],
                                  gamma=2**args['gamma'],
                                  random_state=random_state)

                    if cls_method == "svm" or cls_method == "logit":
                        pipeline = Pipeline([('encoder', feature_combiner), ('scaler', StandardScaler()), ('cls', cls)])
                    else:
                        pipeline = Pipeline([('encoder', feature_combiner), ('cls', cls)])
                    pipeline.fit(dt_train_bucket, train_y)

                    if cls_method == "svm":
                        preds = pipeline.decision_function(dt_test_bucket)
                    else:
                        preds_pos_label_idx = np.where(cls.classes_ == 1)[0][0]
                        preds = pipeline.predict_proba(dt_test_bucket)[:,preds_pos_label_idx]
            
            if "prefix" in method_name:
                auc = 0.5
                if len(set(test_y)) == 2: 
                    auc = roc_auc_score(test_y, preds)
                scores[bucket] += auc
            preds_all.extend(preds)
            test_y_all.extend(test_y)

        score += roc_auc_score(test_y_all, preds_all)
    
    if "prefix" in method_name:
        for k, v in args.items():
            for bucket, bucket_score in scores.items():
                fout_all.write("%s;%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, bucket, k, v, bucket_score / n_splits))   
        fout_all.write("%s;%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, 0, "processing_time", time.time() - start, 0))  
    else:
        for k, v in args.items():
            fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, k, v, score / n_splits))   
        fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, "processing_time", time.time() - start, 0))   
    fout_all.flush()
    return {'loss': -score / n_splits, 'status': STATUS_OK, 'model': cls}


# dataset_ref = argv[1]
# params_dir = argv[2]
# n_iter = int(argv[3])
# bucket_method = argv[4]
# cls_encoding = argv[5]
# cls_method = argv[6]

dataset_ref = "bpic2012"
params_dir = "params"
n_iter = 3
bucket_method = "single"
cls_encoding = "agg"
cls_method = "cb"

if bucket_method == "state":
    bucket_encoding = "last"
else:
    bucket_encoding = "agg"

method_name = "%s_%s"%(bucket_method, cls_encoding)

dataset_ref_to_datasets = {
    "bpic2011": ["bpic2011_f%s"%formula for formula in range(1,5)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(5,6)],
    "insurance": ["insurance_activity", "insurance_followup"],
    "bpic2012" : ["bpic2012_accepted"],
    "sepsis_cases": ["sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4"]
}

encoding_dict = {
    "laststate": ["static", "last"],
    "agg": ["static", "agg"],
    "index": ["static", "index"],
    "combined": ["static", "last", "agg"]
}

datasets = [dataset_ref] if dataset_ref not in dataset_ref_to_datasets else dataset_ref_to_datasets[dataset_ref]
methods = encoding_dict[cls_encoding]
print(datasets)
    
train_ratio = 0.8
n_splits = 3
random_state = 22

# create results directory
if not os.path.exists(os.path.join(params_dir)):
    os.makedirs(os.path.join(params_dir))
    
for dataset_name in datasets:
    
    # read the data
    dataset_manager = DatasetManager(dataset_name)
    data = dataset_manager.read_dataset()
    cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

    # determine min and max (truncated) prefix lengths
    min_prefix_length = 1
    if "traffic_fines" in dataset_name:
        max_prefix_length = 10
    elif "bpic2017" in dataset_name:
        max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    else:
        max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))

    # split into training and test
    print("splitting data")
    train, _ = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
    
    # prepare chunks for CV
    dt_prefixes = []
    class_ratios = []
    for train_chunk, test_chunk in dataset_manager.get_stratified_split_generator(train, n_splits=n_splits):
        class_ratios.append(dataset_manager.get_class_ratio(train_chunk))
        # generate data where each prefix is a separate instance
        dt_prefixes.append(dataset_manager.generate_prefix_data(test_chunk, min_prefix_length, max_prefix_length))
    del train
        
    # set up search space
    if cls_method == "rf":
        space = {'max_features': hp.uniform('max_features', 0, 1)}
    elif cls_method == "xgboost":
        space = {'learning_rate': hp.uniform("learning_rate", 0, 1),
                 'subsample': hp.uniform("subsample", 0.5, 1),
                 'max_depth': scope.int(hp.quniform('max_depth', 4, 30, 1)),
                 'colsample_bytree': hp.uniform("colsample_bytree", 0.5, 1),
                 'min_child_weight': scope.int(hp.quniform('min_child_weight', 1, 6, 1))}
    elif cls_method == "logit":
        space = {'C': hp.uniform('C', -15, 15)}
    elif cls_method == "svm":
        space = {'C': hp.uniform('C', -15, 15),
                 'gamma': hp.uniform('gamma', -15, 15)}
    elif cls_method == "cb":
        space = {'learning_rate': hp.uniform("learning_rate", 0, 1),
                 'depth': scope.int(hp.quniform('max_depth', 4, 16, 1)),
                 'subsample': hp.uniform("subsample", 0.5, 1)}
    if bucket_method == "cluster":
        space['n_clusters'] = scope.int(hp.quniform('n_clusters', 2, 50, 1))

    # optimize parameters
    trial_nr = 1
    trials = Trials()
    fout_all = open(os.path.join(params_dir, "param_optim_all_trials_%s_%s_%s.csv" % (cls_method, dataset_name, method_name)), "w")
    if "prefix" in method_name:
        fout_all.write("%s;%s;%s;%s;%s;%s;%s;%s\n" % ("iter", "dataset", "cls", "method", "nr_events", "param", "value", "score"))   
    else:
        fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % ("iter", "dataset", "cls", "method", "param", "value", "score"))   
    best = fmin(create_and_evaluate_model, space, algo=tpe.suggest, max_evals=n_iter, trials=trials, verbose=True)
    fout_all.close()

    # write the best parameters
    best_params = hyperopt.space_eval(space, best)
    outfile = os.path.join(params_dir, "optimal_params_%s_%s_%s.pickle" % (cls_method, dataset_name, method_name))
    # write to file
    with open(outfile, "wb") as fout:
        pickle.dump(best_params, fout)


['bpic2012_accepted']
splitting data
single                                                                                                                 
0:	learn: 0.6150779	total: 308ms	remaining: 5m 7s                                                                      

1:	learn: 0.5707476	total: 571ms	remaining: 4m 44s                                                                     

2:	learn: 0.5327759	total: 838ms	remaining: 4m 38s                                                                     

3:	learn: 0.4919134	total: 1.12s	remaining: 4m 38s                                                                     

4:	learn: 0.4727505	total: 1.39s	remaining: 4m 35s                                                                     

5:	learn: 0.4470065	total: 1.63s	remaining: 4m 29s                                                                     

6:	learn: 0.4216016	total: 1.88s	remaining: 4m 26s                                                                   

66:	learn: 0.0610603	total: 15.8s	remaining: 3m 40s                                                                    

67:	learn: 0.0598098	total: 16.1s	remaining: 3m 40s                                                                    

68:	learn: 0.0584622	total: 16.3s	remaining: 3m 39s                                                                    

69:	learn: 0.0571070	total: 16.5s	remaining: 3m 39s                                                                    

70:	learn: 0.0552808	total: 16.7s	remaining: 3m 39s                                                                    

71:	learn: 0.0541716	total: 17s	remaining: 3m 38s                                                                      

72:	learn: 0.0536056	total: 17.2s	remaining: 3m 38s                                                                    

73:	learn: 0.0524445	total: 17.4s	remaining: 3m 38s                                                                    

74:	learn: 0.0516563	total: 17.7

133:	learn: 0.0180607	total: 31.1s	remaining: 3m 20s                                                                   

134:	learn: 0.0176999	total: 31.3s	remaining: 3m 20s                                                                   

135:	learn: 0.0175595	total: 31.5s	remaining: 3m 20s                                                                   

136:	learn: 0.0173109	total: 31.8s	remaining: 3m 20s                                                                   

137:	learn: 0.0170236	total: 32s	remaining: 3m 19s                                                                     

138:	learn: 0.0166059	total: 32.2s	remaining: 3m 19s                                                                   

139:	learn: 0.0163973	total: 32.4s	remaining: 3m 19s                                                                   

140:	learn: 0.0161929	total: 32.7s	remaining: 3m 19s                                                                   

141:	learn: 0.0160025	total: 32.

200:	learn: 0.0133194	total: 44.2s	remaining: 2m 55s                                                                   

201:	learn: 0.0128498	total: 44.4s	remaining: 2m 55s                                                                   

202:	learn: 0.0126705	total: 44.6s	remaining: 2m 55s                                                                   

203:	learn: 0.0125449	total: 44.8s	remaining: 2m 54s                                                                   

204:	learn: 0.0123811	total: 45s	remaining: 2m 54s                                                                     

205:	learn: 0.0121955	total: 45.2s	remaining: 2m 54s                                                                   

206:	learn: 0.0120321	total: 45.5s	remaining: 2m 54s                                                                   

207:	learn: 0.0119353	total: 45.7s	remaining: 2m 53s                                                                   

208:	learn: 0.0119353	total: 45.

267:	learn: 0.0076022	total: 57.9s	remaining: 2m 38s                                                                   

268:	learn: 0.0076022	total: 58s	remaining: 2m 37s                                                                     

269:	learn: 0.0076022	total: 58.2s	remaining: 2m 37s                                                                   

270:	learn: 0.0076020	total: 58.4s	remaining: 2m 37s                                                                   

271:	learn: 0.0076019	total: 58.6s	remaining: 2m 36s                                                                   

272:	learn: 0.0076017	total: 58.8s	remaining: 2m 36s                                                                   

273:	learn: 0.0076016	total: 58.9s	remaining: 2m 36s                                                                   

274:	learn: 0.0075845	total: 59.1s	remaining: 2m 35s                                                                   

275:	learn: 0.0075845	total: 59.

...

701:	learn: 0.0031928	total: 55m 52s	remaining: 23m 43s                                                                

702:	learn: 0.0031928	total: 55m 57s	remaining: 23m 38s                                                                

703:	learn: 0.0031927	total: 56m 1s	remaining: 23m 33s                                                                 

704:	learn: 0.0031927	total: 56m 6s	remaining: 23m 28s                                                                 

705:	learn: 0.0031927	total: 56m 10s	remaining: 23m 23s                                                                

706:	learn: 0.0031925	total: 56m 15s	remaining: 23m 18s                                                                

707:	learn: 0.0031924	total: 56m 19s	remaining: 23m 14s                                                                

708:	learn: 0.0031924	total: 56m 24s	remaining: 23m 9s                                                                 

709:	learn: 0.0031924	total: 56m

768:	learn: 0.0031903	total: 1h 52s	remaining: 18m 17s                                                                 

769:	learn: 0.0031903	total: 1h 56s	remaining: 18m 12s                                                                 

770:	learn: 0.0031903	total: 1h 1m	remaining: 18m 7s                                                                   

771:	learn: 0.0031903	total: 1h 1m 5s	remaining: 18m 2s                                                                

772:	learn: 0.0031902	total: 1h 1m 10s	remaining: 17m 57s                                                              

773:	learn: 0.0031902	total: 1h 1m 14s	remaining: 17m 52s                                                              

774:	learn: 0.0031902	total: 1h 1m 18s	remaining: 17m 48s                                                              

775:	learn: 0.0031900	total: 1h 1m 23s	remaining: 17m 43s                                                              

776:	learn: 0.0031900	total: 1h 

835:	learn: 0.0031875	total: 1h 5m 53s	remaining: 12m 55s                                                              

836:	learn: 0.0031875	total: 1h 5m 58s	remaining: 12m 50s                                                              

837:	learn: 0.0031875	total: 1h 6m 2s	remaining: 12m 46s                                                               

838:	learn: 0.0031875	total: 1h 6m 6s	remaining: 12m 41s                                                               

839:	learn: 0.0031875	total: 1h 6m 11s	remaining: 12m 36s                                                              

840:	learn: 0.0031875	total: 1h 6m 15s	remaining: 12m 31s                                                              

841:	learn: 0.0031875	total: 1h 6m 19s	remaining: 12m 26s                                                              

842:	learn: 0.0031874	total: 1h 6m 24s	remaining: 12m 22s                                                              

843:	learn: 0.0031874	total: 1h 

902:	learn: 0.0031070	total: 1h 10m 56s	remaining: 7m 37s                                                              

903:	learn: 0.0031070	total: 1h 11m	remaining: 7m 32s                                                                  

904:	learn: 0.0031069	total: 1h 11m 5s	remaining: 7m 27s                                                               

905:	learn: 0.0031069	total: 1h 11m 9s	remaining: 7m 23s                                                               

906:	learn: 0.0031069	total: 1h 11m 14s	remaining: 7m 18s                                                              

907:	learn: 0.0031068	total: 1h 11m 18s	remaining: 7m 13s                                                              

908:	learn: 0.0031068	total: 1h 11m 23s	remaining: 7m 8s                                                               

909:	learn: 0.0031067	total: 1h 11m 27s	remaining: 7m 4s                                                               

910:	learn: 0.0031067	total: 1h 

969:	learn: 0.0030498	total: 1h 16m	remaining: 2m 21s                                                                  

970:	learn: 0.0030497	total: 1h 16m 4s	remaining: 2m 16s                                                               

971:	learn: 0.0030497	total: 1h 16m 9s	remaining: 2m 11s                                                               

972:	learn: 0.0030497	total: 1h 16m 13s	remaining: 2m 6s                                                               

973:	learn: 0.0030496	total: 1h 16m 18s	remaining: 2m 2s                                                               

974:	learn: 0.0030496	total: 1h 16m 22s	remaining: 1m 57s                                                              

975:	learn: 0.0030496	total: 1h 16m 27s	remaining: 1m 52s                                                              

976:	learn: 0.0030496	total: 1h 16m 31s	remaining: 1m 48s                                                              

977:	learn: 0.0030496	total: 1h 